In [1]:
!pip install scikeras

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, precision_score, recall_score, f1_score
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.model_selection import  GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
import matplotlib.pyplot as plt
import warnings
import time
import gc
import datetime

pd.set_option('display.float_format', lambda x: '%.3f' %x)
warnings.filterwarnings("ignore")

In [3]:
##### Memory reduction script 1

def sd(col, max_loss_limit=0.001, avg_loss_limit=0.001, na_loss_limit=0, n_uniq_loss_limit=0, fillna=0):
    """
    max_loss_limit - don't allow any float to lose precision more than this value. Any values are ok for GBT algorithms as long as you don't unique values.
                     See https://en.wikipedia.org/wiki/Half-precision_floating-point_format#Precision_limitations_on_decimal_values_in_[0,_1]
    avg_loss_limit - same but calculates avg throughout the series.
    na_loss_limit - not really useful.
    n_uniq_loss_limit - very important parameter. If you have a float field with very high cardinality you can set this value to something like n_records * 0.01 in order to allow some field relaxing.
    """
    is_float = str(col.dtypes)[:5] == 'float'
    na_count = col.isna().sum()
    n_uniq = col.nunique(dropna=False)
    try_types = ['float16', 'float32']

    if na_count <= na_loss_limit:
        try_types = ['int8', 'int16', 'float16', 'int32', 'float32']

    for type in try_types:
        col_tmp = col

        # float to int conversion => try to round to minimize casting error
        if is_float and (str(type)[:3] == 'int'):
            col_tmp = col_tmp.copy().fillna(fillna).round()

        col_tmp = col_tmp.astype(type)
        max_loss = (col_tmp - col).abs().max()
        avg_loss = (col_tmp - col).abs().mean()
        na_loss = np.abs(na_count - col_tmp.isna().sum())
        n_uniq_loss = np.abs(n_uniq - col_tmp.nunique(dropna=False))

        if max_loss <= max_loss_limit and avg_loss <= avg_loss_limit and na_loss <= na_loss_limit and n_uniq_loss <= n_uniq_loss_limit:
            return col_tmp

    # field can't be converted
    return col

##### Memory reduction script 2

def reduce_mem_usage_sd(df, deep=True, verbose=False, obj_to_cat=False):
    numerics = ['int16', 'uint16', 'int32', 'uint32', 'int64', 'uint64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=deep).sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes

        # collect stats
        na_count = df[col].isna().sum()
        n_uniq = df[col].nunique(dropna=False)

        # numerics
        if col_type in numerics:
            df[col] = sd(df[col])

        # strings
        if (col_type == 'object') and obj_to_cat:
            df[col] = df[col].astype('category')

        if verbose:
            print(f'Column {col}: {col_type} -> {df[col].dtypes}, na_count={na_count}, n_uniq={n_uniq}')
        new_na_count = df[col].isna().sum()
        if (na_count != new_na_count):
            print(f'Warning: column {col}, {col_type} -> {df[col].dtypes} lost na values. Before: {na_count}, after: {new_na_count}')
        new_n_uniq = df[col].nunique(dropna=False)
        if (n_uniq != new_n_uniq):
            print(f'Warning: column {col}, {col_type} -> {df[col].dtypes} lost unique values. Before: {n_uniq}, after: {new_n_uniq}')

    end_mem = df.memory_usage(deep=deep).sum() / 1024 ** 2
    percent = 100 * (start_mem - end_mem) / start_mem
    if verbose:
        print('Mem. usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(start_mem, end_mem, percent))
    return df

##### Memory reduction script 3
# https://www.kaggle.com/kyakovlev/ieee-data-minification
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024 ** 2 # just added
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage(deep=True).sum() / 1024 ** 2
    percent = 100 * (start_mem - end_mem) / start_mem
    print('Mem. usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(start_mem, end_mem, percent))
    return df

##### Functions
# 1st function: to graph time series based on TransactionDT vs the variable selected

def scatter(column):
    fr,no_fr = (train[train['isFraud'] == 1], train[train['isFraud'] == 0])
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,3))
    ax1.title.set_text('Histogram ' + column + ' when isFraud == 0')
    ax1.set_ylim(train[column].min() - 1,train[column].max() + 1)
    ax1.scatter(x = no_fr['TransactionDT'], y = no_fr[column], color = 'blue', marker='o')
    ax2.title.set_text('Histogram ' + column + ' when isFraud == 1')
    ax2.set_ylim(train[column].min() - 1,train[column].max() + 1)
    ax2.scatter(x = fr['TransactionDT'], y = fr[column], color = 'red', marker='o')
    plt.show()

# 2nd function: to show a ranking of pearson correlation with the variable selected

def corr(data,column):
    print('Correlation with ' + column)
    print(train[data].corrwith(data[column]).abs().sort_values(ascending = False)[1:])

# 3rd function: to reduce the groups based on Nans agroupation and pearson correlation

def reduce(groups):
    result = list()
    for values in groups:
        maxval = 0
        val = values[0]
        for value in values:
            unique_values = data[value].nunique()
            if unique_values > maxval:
                maxval = unique_values
                val = value
        result.append(value)
    return result

# 4th function: to sort each column in ascending order based on its number

def order_finalcolumns(final_Xcolumns):
    return sorted(final_Xcolumns, key=lambda x: int("".join([i for i in x if i.isdigit()])))


In [4]:
#Carregando os dados
train_identity = pd.read_csv('train_identity.csv')
train_transaction = pd.read_csv('train_transaction.csv')

#Efetuar o merge pela coluna TransactionID
data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')


In [5]:
%%time
#Tratamento para melhorar a coluna DeviceInfo
data.loc[data['DeviceInfo'].str.contains('SM', na=False), 'DeviceInfo'] = 'Samsung'
data.loc[data['DeviceInfo'].str.contains('SAMSUNG', na=False), 'DeviceInfo'] = 'Samsung'
data.loc[data['DeviceInfo'].str.contains('GT-', na=False), 'DeviceInfo'] = 'Samsung'
data.loc[data['DeviceInfo'].str.contains('Moto G', na=False), 'DeviceInfo'] = 'Motorola'
data.loc[data['DeviceInfo'].str.contains('Moto', na=False), 'DeviceInfo'] = 'Motorola'
data.loc[data['DeviceInfo'].str.contains('moto', na=False), 'DeviceInfo'] = 'Motorola'
data.loc[data['DeviceInfo'].str.contains('LG-', na=False), 'DeviceInfo'] = 'LG'
data.loc[data['DeviceInfo'].str.contains('rv:', na=False), 'DeviceInfo'] = 'RV'
data.loc[data['DeviceInfo'].str.contains('HUAWEI', na=False), 'DeviceInfo'] = 'Huawei'
data.loc[data['DeviceInfo'].str.contains('ALE-', na=False), 'DeviceInfo'] = 'Huawei'
data.loc[data['DeviceInfo'].str.contains('-L', na=False), 'DeviceInfo'] = 'Huawei'
data.loc[data['DeviceInfo'].str.contains('Blade', na=False), 'DeviceInfo'] = 'ZTE'
data.loc[data['DeviceInfo'].str.contains('BLADE', na=False), 'DeviceInfo'] = 'ZTE'
data.loc[data['DeviceInfo'].str.contains('Linux', na=False), 'DeviceInfo'] = 'Linux'
data.loc[data['DeviceInfo'].str.contains('XT', na=False), 'DeviceInfo'] = 'Sony'
data.loc[data['DeviceInfo'].str.contains('HTC', na=False), 'DeviceInfo'] = 'HTC'
data.loc[data['DeviceInfo'].str.contains('ASUS', na=False), 'DeviceInfo'] = 'Asus'
#Tratamento para unificar e-mail por provider.
data.loc[data['P_emaildomain'].str.contains('gmail', na=False), 'P_emaildomain'] = 'Google'
data.loc[data['P_emaildomain'].str.contains('hotmail|outlook|msn|live|', na=False), 'P_emaildomain'] = 'Microsoft'
data.loc[data['P_emaildomain'].str.contains('yahoo|ymail|att|bellsouth', na=False), 'P_emaildomain'] = 'Yahoo'
data.loc[data['P_emaildomain'].str.contains('icloud|me|mac', na=False), 'P_emaildomain'] = 'Apple'
data.loc[data['P_emaildomain'].str.contains('aol|aim', na=False), 'P_emaildomain'] = 'AOL'
data.loc[data['P_emaildomain'].str.contains('embarqmail|centurylink', na=False), 'P_emaildomain'] = 'CenturyLink'
data.loc[data['P_emaildomain'].str.contains('frontier', na=False), 'P_emaildomain'] = 'Frontier'
data.loc[data['P_emaildomain'].str.contains('netzero', na=False), 'P_emaildomain'] = 'Netzero'
data.loc[data['P_emaildomain'].str.contains('cfl|sc.rr|twc', na=False), 'P_emaildomain'] = 'Spectrum'
data.loc[data['P_emaildomain'].str.contains('protonmail|ptd|servicios|roadrunner|juno|windstream|scranton', na=False), 'P_emaildomain'] = 'Others'
data.loc[data['P_emaildomain'].str.contains('q.com|web|suddenlink|cableone|prodigy|gmx', na=False), 'P_emaildomain'] = 'Others'

data.loc[data['R_emaildomain'].str.contains('gmail', na=False), 'R_emaildomain'] = 'Google'
data.loc[data['R_emaildomain'].str.contains('hotmail|outlook|msn|live|', na=False), 'R_emaildomain'] = 'Microsoft'
data.loc[data['R_emaildomain'].str.contains('yahoo|ymail|att|bellsouth', na=False), 'R_emaildomain'] = 'Yahoo'
data.loc[data['R_emaildomain'].str.contains('icloud|me|mac', na=False), 'R_emaildomain'] = 'Apple'
data.loc[data['R_emaildomain'].str.contains('aol|aim', na=False), 'R_emaildomain'] = 'AOL'
data.loc[data['R_emaildomain'].str.contains('embarqmail|centurylink', na=False), 'R_emaildomain'] = 'CenturyLink'
data.loc[data['R_emaildomain'].str.contains('frontier', na=False), 'R_emaildomain'] = 'Frontier'
data.loc[data['R_emaildomain'].str.contains('netzero', na=False), 'R_emaildomain'] = 'Netzero'
data.loc[data['R_emaildomain'].str.contains('cfl|sc.rr|twc', na=False), 'R_emaildomain'] = 'Spectrum'
data.loc[data['R_emaildomain'].str.contains('protonmail|ptd|servicios|roadrunner|juno|windstream|scranton', na=False), 'R_emaildomain'] = 'Others'
data.loc[data['R_emaildomain'].str.contains('q.com|web|suddenlink|cableone|prodigy|gmx', na=False), 'R_emaildomain'] = 'Others'

#Temos o TransactionDT que é uma data aleatorio no tempo, irei transformar em Dia,Mes e Ano para facilitar a analise.
#data['TransactionDT'] = pd.to_datetime(data['TransactionDT']) - Não ficou bom.

# Converter a coluna de datas para componentes numéricos
startdate = datetime.datetime.strptime('2017-12-01', '%Y-%m-%d')
data['TransactionDT'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
data.insert(4, 'TransHour', data['TransactionDT'].dt.hour)
data.insert(5, 'TransDay', data['TransactionDT'].dt.day)
data.insert(6, 'TransMonth', data['TransactionDT'].dt.month)
data.insert(7, 'TransWeek', data['TransactionDT'].dt.isocalendar().week)
data = data.drop(columns=['TransactionDT'])

CPU times: total: 6.3 s
Wall time: 10.1 s


In [6]:
#data = data.replace(np.nan, 0)
#Efetuo redução de memoria.
data = reduce_mem_usage_sd(data, verbose=True)

Column TransactionID: int64 -> int32, na_count=0, n_uniq=590540
Column isFraud: int64 -> int8, na_count=0, n_uniq=2
Column TransactionAmt: float64 -> float32, na_count=0, n_uniq=20902
Column TransHour: int64 -> int8, na_count=0, n_uniq=24
Column TransDay: int64 -> int8, na_count=0, n_uniq=31
Column TransMonth: int64 -> int8, na_count=0, n_uniq=7
Column TransWeek: UInt32 -> UInt32, na_count=0, n_uniq=27
Column ProductCD: object -> object, na_count=0, n_uniq=5
Column card1: int64 -> int16, na_count=0, n_uniq=13553
Column card2: float64 -> float16, na_count=8933, n_uniq=501
Column card3: float64 -> float16, na_count=1565, n_uniq=115
Column card4: object -> object, na_count=1577, n_uniq=5
Column card5: float64 -> float16, na_count=4259, n_uniq=120
Column card6: object -> object, na_count=1571, n_uniq=5
Column addr1: float64 -> float16, na_count=65706, n_uniq=333
Column addr2: float64 -> float16, na_count=65706, n_uniq=75
Column dist1: float64 -> float32, na_count=352271, n_uniq=2652
Column

Column V86: float64 -> float16, na_count=89164, n_uniq=32
Column V87: float64 -> float16, na_count=89164, n_uniq=32
Column V88: float64 -> float16, na_count=89164, n_uniq=3
Column V89: float64 -> float16, na_count=89164, n_uniq=4
Column V90: float64 -> float16, na_count=89164, n_uniq=7
Column V91: float64 -> float16, na_count=89164, n_uniq=8
Column V92: float64 -> float16, na_count=89164, n_uniq=9
Column V93: float64 -> float16, na_count=89164, n_uniq=9
Column V94: float64 -> float16, na_count=89164, n_uniq=4
Column V95: float64 -> float16, na_count=314, n_uniq=882
Column V96: float64 -> float16, na_count=314, n_uniq=1411
Column V97: float64 -> float16, na_count=314, n_uniq=977
Column V98: float64 -> float16, na_count=314, n_uniq=14
Column V99: float64 -> float16, na_count=314, n_uniq=90
Column V100: float64 -> float16, na_count=314, n_uniq=30
Column V101: float64 -> float16, na_count=314, n_uniq=871
Column V102: float64 -> float16, na_count=314, n_uniq=1286
Column V103: float64 -> flo

Column V224: float64 -> float16, na_count=460110, n_uniq=80
Column V225: float64 -> float16, na_count=460110, n_uniq=36
Column V226: float64 -> float16, na_count=460110, n_uniq=82
Column V227: float64 -> float16, na_count=449124, n_uniq=51
Column V228: float64 -> float16, na_count=460110, n_uniq=56
Column V229: float64 -> float16, na_count=460110, n_uniq=92
Column V230: float64 -> float16, na_count=460110, n_uniq=67
Column V231: float64 -> float16, na_count=460110, n_uniq=295
Column V232: float64 -> float16, na_count=460110, n_uniq=339
Column V233: float64 -> float16, na_count=460110, n_uniq=334
Column V234: float64 -> float16, na_count=449124, n_uniq=123
Column V235: float64 -> float16, na_count=460110, n_uniq=25
Column V236: float64 -> float16, na_count=460110, n_uniq=47
Column V237: float64 -> float16, na_count=460110, n_uniq=41
Column V238: float64 -> float16, na_count=449124, n_uniq=25
Column V239: float64 -> float16, na_count=449124, n_uniq=25
Column V240: float64 -> float16, na_

Column id_24: float64 -> float16, na_count=585793, n_uniq=13
Column id_25: float64 -> float16, na_count=585408, n_uniq=342
Column id_26: float64 -> float16, na_count=585377, n_uniq=96
Column id_27: object -> object, na_count=585371, n_uniq=3
Column id_28: object -> object, na_count=449562, n_uniq=3
Column id_29: object -> object, na_count=449562, n_uniq=3
Column id_30: object -> object, na_count=512975, n_uniq=76
Column id_31: object -> object, na_count=450258, n_uniq=131
Column id_32: float64 -> float16, na_count=512954, n_uniq=5
Column id_33: object -> object, na_count=517251, n_uniq=261
Column id_34: object -> object, na_count=512735, n_uniq=5
Column id_35: object -> object, na_count=449555, n_uniq=3
Column id_36: object -> object, na_count=449555, n_uniq=3
Column id_37: object -> object, na_count=449555, n_uniq=3
Column id_38: object -> object, na_count=449555, n_uniq=3
Column DeviceType: object -> object, na_count=449730, n_uniq=3
Column DeviceInfo: object -> object, na_count=4718

In [7]:
columns= list(data.columns[:20])
columns

['TransactionID',
 'isFraud',
 'TransactionAmt',
 'TransHour',
 'TransDay',
 'TransMonth',
 'TransWeek',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'P_emaildomain',
 'R_emaildomain']

In [8]:
for col in columns:
    print(f'{col} NaNs: {data[col].isna().sum()} | {data[col].isna().sum()/data.shape[0]:.2%}')

TransactionID NaNs: 0 | 0.00%
isFraud NaNs: 0 | 0.00%
TransactionAmt NaNs: 0 | 0.00%
TransHour NaNs: 0 | 0.00%
TransDay NaNs: 0 | 0.00%
TransMonth NaNs: 0 | 0.00%
TransWeek NaNs: 0 | 0.00%
ProductCD NaNs: 0 | 0.00%
card1 NaNs: 0 | 0.00%
card2 NaNs: 8933 | 1.51%
card3 NaNs: 1565 | 0.27%
card4 NaNs: 1577 | 0.27%
card5 NaNs: 4259 | 0.72%
card6 NaNs: 1571 | 0.27%
addr1 NaNs: 65706 | 11.13%
addr2 NaNs: 65706 | 11.13%
dist1 NaNs: 352271 | 59.65%
dist2 NaNs: 552913 | 93.63%
P_emaildomain NaNs: 94456 | 15.99%
R_emaildomain NaNs: 453249 | 76.75%


In [9]:
# If we look closely to % NaNs data, most of them have low number of missing information. We are keeping all the columns where % NaNs < 0.7

final_transactioncolumns = list()
for col in columns:
    if data[col].isna().sum()/data.shape[0] < 0.7:
        final_transactioncolumns.append(col)
print('Final Transaction columns:',final_transactioncolumns)

Final Transaction columns: ['TransactionID', 'isFraud', 'TransactionAmt', 'TransHour', 'TransDay', 'TransMonth', 'TransWeek', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'P_emaildomain']


In [10]:
##### Based on pearson correlation, we grouped together the columns with corr > 0.7

reduce_groups = [['C1','C11','C2','C6','C8','C4','C10','C14','C12','C7','C13'], ['C3'], ['C5','C9']]

result = reduce(reduce_groups)
print('Final C columns:',result)
final_ccolumns = result


Final C columns: ['C13', 'C3', 'C9']


In [11]:
reduce_groups = [['D3','D7','D5'], ['D8'], ['D9']]

result = reduce(reduce_groups)
print('Final D columns:',result)
final_dcolumns = result

Final D columns: ['D5', 'D8', 'D9']


In [12]:
final_vcolumns = list()
reduce_groups = ['V1'], ['V2','V3'], ['V4','V5'], ['V6','V7'], ['V8','V9']

result = reduce(reduce_groups)
final_vcolumns.extend(result)

In [13]:
reduce_groups = [['V12','V13'], ['V14'], ['V15','V16','V33','V34','V31','V32','V21','V22','V17','V18'], ['V19','V20'],['V23','V24'],['V25','V26'],['V27','V28'],['V29','V30']]

result = reduce(reduce_groups)
final_vcolumns.extend(result)

In [14]:
reduce_groups = [['V35','V36'], ['V37','V38'], ['V39','V40','V42','V43','V50','V51','V52'], ['V41'], ['V44','V45'],['V46','V47'],['V48','V49']]

result = reduce(reduce_groups)
final_vcolumns.extend(result)

In [15]:
reduce_groups = [['V53','V54'], ['V55','V56'], ['V57','V58','V71','V73','V72','V74','V63','V59','V64','V60'],['V61','V62'],['V65'],
                ['V66','V67'],['V68'], ['V69','V70']]

result = reduce(reduce_groups)
final_vcolumns.extend(result)
print('Final V_Group4 columns:',result)

Final V_Group4 columns: ['V54', 'V56', 'V60', 'V62', 'V65', 'V67', 'V68', 'V70']


In [16]:
reduce_groups = [['V95','V101'],['V96','V102','V97','V99','V100','V103'],['V98'],['V104','V106','V105'],['V108','V110','V114','V109','V111','V113','V112','V115','V116'],
                ['V117','V119','V118'],['V120','V122','V121'],['V123','V125','V124'],['V126','V128','V132'],['V127','V133','V134'],['V129','V131','V130'],
                ['V135','V137','V136']]

result = reduce(reduce_groups)
final_vcolumns.extend(result)
print('Final V_Group6 columns:',result)

Final V_Group6 columns: ['V101', 'V103', 'V98', 'V105', 'V116', 'V118', 'V121', 'V124', 'V132', 'V134', 'V130', 'V136']


In [17]:
reduce_groups = [['V138'],['V139','V140'],['V141','V142'],['V143','V159','V150','V151','V165','V144','V145','V160','V152','V164','V166'],['V146','V147'],
                ['V148','V155','V149','V153','V154','V156','V157','V158'],['V161','V163','V162']]

result = reduce(reduce_groups)
final_vcolumns.extend(result)
print('Final V_Group7 columns:',result)

Final V_Group7 columns: ['V138', 'V140', 'V142', 'V166', 'V147', 'V158', 'V162']


In [18]:
reduce_groups = ['V167','V176','V199','V179','V190','V177','V186','V168','V172','V178','V196','V191','V204','V213','V207','V173'],['V181','V183','V182',
                'V187','V192','V203','V215','V178','V193','V212','V204'],['V202','V216','V204','V214']

result = reduce(reduce_groups)
final_vcolumns.extend(result)
print('Final V_Group8 columns:',result)

Final V_Group8 columns: ['V173', 'V204', 'V214']


In [19]:
reduce_groups = [['V169'],['V170','V171','V200','V201'],['V174','V175'],['V180'],['V184','V185'],['V188','V189'],['V194','V197','V195','V198'],
                ['V208','V210','V209']]

result = reduce(reduce_groups)
final_vcolumns.extend(result)
print('Final V_Group9 columns:',result)

Final V_Group9 columns: ['V169', 'V201', 'V175', 'V180', 'V185', 'V189', 'V198', 'V209']


In [20]:
reduce_groups = [['V217','V231','V233','V228','V257','V219','V232','V246'],['V218','V229','V224','V225','V253','V243','V254','V248','V264','V261','V249','V258',
                'V267','V274','V230','V236','V247','V262','V223','V252','V260'],['V226','V263','V276','V278'], ['V235','V237'],['V240','V241'],['V242','V244'],
                ['V265','V275','V277','V268','V273'],['V269','V266']]

result = reduce(reduce_groups)
final_vcolumns.extend(result)
print('Final V_Group10 columns:',result)

Final V_Group10 columns: ['V246', 'V260', 'V278', 'V237', 'V241', 'V244', 'V273', 'V266']


In [21]:
reduce_groups = ['V220'],['V221','V222','V259','V245','V227','V255','V256'],['V234'],['V238','V239'],['V250','V251'],['V270','V272','V271']

result = reduce(reduce_groups)
final_vcolumns.extend(result)
print('Final V_Group11 columns:',result)

Final V_Group11 columns: ['V220', 'V256', 'V234', 'V239', 'V251', 'V271']


In [22]:
reduce_groups = [['V279','V293','V290','V280','V295','V294','V292','V291','V317','V307','V318'],['V284'],['V285','V287'],['V286'],['V297','V299','V298'],
                ['V302','V304','V303'],['V305'],['V306','V308','V316','V319'],['V309','V311','V312','V310'],['V320','V321']]

result = reduce(reduce_groups)
final_vcolumns.extend(result)
print('Final V_Group12 columns:',result)

Final V_Group12 columns: ['V318', 'V284', 'V287', 'V286', 'V298', 'V303', 'V305', 'V319', 'V310', 'V321']


In [23]:
reduce_groups = ['V281','V282','V283'],['V288','V289'],['V296'],['V300','V301'],['V313','V315','V314']

result = reduce(reduce_groups)
final_vcolumns.extend(result)
print('Final V_Group13 columns:',result)

Final V_Group13 columns: ['V283', 'V289', 'V296', 'V301', 'V314']


In [24]:
reduce_groups = ['V322','V324'],['V323','V326','V324','V327','V326'],['V325'],['V328','V330','V329'],['V331','V333','V332','V337'],['V334','V336','V335']

result = reduce(reduce_groups)
final_vcolumns.extend(result)
print('Final V_Group14 columns:',result)

Final V_Group14 columns: ['V324', 'V326', 'V325', 'V329', 'V337', 'V335']


In [25]:
#Utilizei uma estrategia de agrupamento por correlação de Person, quando > 0.7, para tentar maximizar o dataset
reduce_groups = ['M1'], ['M2','M3'], ['M4'], ['M5'], ['M6'], ['M7', 'M8'], ['M9']

result = reduce(reduce_groups)
print('Final M columns:',result)
final_mcolumns = result

Final M columns: ['M1', 'M3', 'M4', 'M5', 'M6', 'M8', 'M9']


In [26]:
final_ccolumns = order_finalcolumns(final_ccolumns)
final_dcolumns = order_finalcolumns(final_dcolumns)
final_mcolumns = order_finalcolumns(final_mcolumns)
final_vcolumns = order_finalcolumns(final_vcolumns)

In [27]:
##### Final columns

print(f'Final Transaction columns ({len(final_transactioncolumns)}): {final_transactioncolumns}')
print(' ')
print(f'Final C columns ({len(final_ccolumns)}): {final_ccolumns}')
print(' ')
print(f'Final D columns ({len(final_dcolumns)}): {final_dcolumns}')
print(' ')
print(f'Final M columns ({len(final_mcolumns)}): {final_mcolumns}')
print(' ')
print(f'Final V columns ({len(final_vcolumns)}): {final_vcolumns}')
print(' ')

print('#' * 50)

final_columns = final_transactioncolumns + final_ccolumns + final_dcolumns + final_mcolumns + final_vcolumns
print(' ')
print('Final columns:', final_columns)
print(' ')
print('Lenght of final columns:', len(final_columns))

Final Transaction columns (18): ['TransactionID', 'isFraud', 'TransactionAmt', 'TransHour', 'TransDay', 'TransMonth', 'TransWeek', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'P_emaildomain']
 
Final C columns (3): ['C3', 'C9', 'C13']
 
Final D columns (3): ['D5', 'D8', 'D9']
 
Final M columns (7): ['M1', 'M3', 'M4', 'M5', 'M6', 'M8', 'M9']
 
Final V columns (93): ['V1', 'V3', 'V5', 'V7', 'V9', 'V13', 'V14', 'V18', 'V20', 'V24', 'V26', 'V28', 'V30', 'V36', 'V38', 'V41', 'V45', 'V47', 'V49', 'V52', 'V54', 'V56', 'V60', 'V62', 'V65', 'V67', 'V68', 'V70', 'V98', 'V101', 'V103', 'V105', 'V116', 'V118', 'V121', 'V124', 'V130', 'V132', 'V134', 'V136', 'V138', 'V140', 'V142', 'V147', 'V158', 'V162', 'V166', 'V169', 'V173', 'V175', 'V180', 'V185', 'V189', 'V198', 'V201', 'V204', 'V209', 'V214', 'V220', 'V234', 'V237', 'V239', 'V241', 'V244', 'V246', 'V251', 'V256', 'V260', 'V266', 'V271', 'V273', 'V278', 'V283', 'V284', 'V286', 'V287', 'V289', 

In [28]:
#data2 = data
# Define o tamanho do subconjunto
#n_samples = 80000

# Realiza a amostragem aleatória
#df_sample = data.sample(n_samples)
#data = df_sample

In [29]:
dddddata

NameError: name 'dddddata' is not defined

In [ ]:
%%time
data.fillna('UNK')
labelencoder = LabelEncoder()
for col in data.columns:
        data[col] = labelencoder.fit_transform(data[col].astype(str))

In [30]:
#del data
df_filtrado = data[final_columns]

In [32]:
colunas = ['TransactionID', 'isFraud', 'TransactionAmt', 'TransHour', 'TransDay', 'TransMonth', 'TransWeek', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'P_emaildomain', 'C3', 'C9', 'C13', 'D5', 'D8', 'D9', 'M1', 'M3', 'M4', 'M5', 'M6', 'M8', 'M9', 'V1', 'V3', 'V5', 'V7', 'V9', 'V13', 'V14', 'V18', 'V20', 'V24', 'V26', 'V28', 'V30', 'V36', 'V38', 'V41', 'V45', 'V47', 'V49', 'V52', 'V54', 'V56', 'V60', 'V62', 'V65', 'V67', 'V68', 'V70', 'V98', 'V101', 'V103', 'V105', 'V116', 'V118', 'V121', 'V124', 'V130', 'V132', 'V134', 'V136', 'V138', 'V140', 'V142', 'V147', 'V158', 'V162', 'V166', 'V169', 'V173', 'V175', 'V180', 'V185', 'V189', 'V198', 'V201', 'V204', 'V209', 'V214', 'V220', 'V234', 'V237', 'V239', 'V241', 'V244', 'V246', 'V251', 'V256', 'V260', 'V266', 'V271', 'V273', 'V278', 'V283', 'V284', 'V286', 'V287', 'V289', 'V296', 'V298', 'V301', 'V303', 'V305', 'V310', 'V314', 'V318', 'V319', 'V321', 'V324', 'V325', 'V326', 'V329', 'V335', 'V337']

# Imprimir as colunas em linhas
print("Colunas Finais:")
print("\n".join(colunas))


Colunas Finais:
TransactionID
isFraud
TransactionAmt
TransHour
TransDay
TransMonth
TransWeek
ProductCD
card1
card2
card3
card4
card5
card6
addr1
addr2
dist1
P_emaildomain
C3
C9
C13
D5
D8
D9
M1
M3
M4
M5
M6
M8
M9
V1
V3
V5
V7
V9
V13
V14
V18
V20
V24
V26
V28
V30
V36
V38
V41
V45
V47
V49
V52
V54
V56
V60
V62
V65
V67
V68
V70
V98
V101
V103
V105
V116
V118
V121
V124
V130
V132
V134
V136
V138
V140
V142
V147
V158
V162
V166
V169
V173
V175
V180
V185
V189
V198
V201
V204
V209
V214
V220
V234
V237
V239
V241
V244
V246
V251
V256
V260
V266
V271
V273
V278
V283
V284
V286
V287
V289
V296
V298
V301
V303
V305
V310
V314
V318
V319
V321
V324
V325
V326
V329
V335
V337


In [ ]:

# Separate input features and output feature
X_train = df_filtrado.drop(columns = ['isFraud'])       # input features
y_train  = df_filtrado.isFraud      # output feature

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
from sklearn.decomposition import PCA

# Crie uma instância do PCA e especifique a quantidade de variância explicada desejada (por exemplo, 0.95)
pca = PCA(n_components=0.95, random_state=0)

# Ajuste o PCA aos seus dados escalados
principalComponents = pca.fit_transform(X_train)

# Crie um DataFrame com os componentes principais
principalDF = pd.DataFrame(data=principalComponents, columns=[f"PC{x}" for x in range(1, principalComponents.shape[1] + 1)])


In [ ]:
principalDF

In [ ]:
# Calcular a taxa de variância explicada cumulativa
explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plotar a taxa de variância explicada
plt.figure(figsize=(8, 4))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Variance Explained')
plt.xlabel('Número de Componentes Principais')
plt.ylabel('Taxa de Variância Explicada Cumulativa')
plt.title('Análise de PCA - Taxa de Variância Explicada')
plt.grid(True)
plt.legend()
plt.show()

# Selecionar as componentes principais automaticamente com n_components=0.95
print(f"Número de componentes principais selecionados: {pca.n_components_}")

In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(principalComponents, y_train)


In [ ]:
del df_filtrado, data, train_identity, train_transaction,principalDF
gc.collect()

In [ ]:
# Definindo o modelo de rede neural
def create_model(input_shape, learning_rate=0.001):
  # Defina a nova forma de entrada com base na dimensionalidade após a PCA
    input_shape = (41,)
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Função para calcular métricas em cada dobra
def calculate_metrics(y_true, y_pred):
    auc = roc_auc_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    cf = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return auc, acc, cf, precision, recall, f1


In [ ]:
from scikeras.wrappers import KerasClassifier

# Especificando a forma de entrada com base no número de recursos em seus dados de entrada
input_shape = (X_train_resampled.shape[1],)

# Criando o classificador Keras
model = KerasClassifier(build_fn=create_model, verbose=0, input_shape=input_shape)

# Aplicando estratificação k-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Definindo os parâmetros para busca em grade
param_grid = {
    'epochs': [5,10],
    'batch_size': [32,64,128]
}

# Listas para armazenar métricas em cada dobra
auc_scores = []
accuracy_scores = []
confusion_matrices = []
precision_scores = []
recall_scores = []
f1_scores = []

In [ ]:
# Listas para armazenar métricas em cada dobra
results = []

# Loop sobre as dobras da validação cruzada
for train_idx, val_idx in kfold.split(X_train_resampled, y_train_resampled):
    X_train_fold, X_val_fold = X_train_resampled[train_idx], X_train_resampled[val_idx]
    y_train_fold, y_val_fold = y_train_resampled[train_idx], y_train_resampled[val_idx]


    # Realizando busca aleatória de hiperparâmetros
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                                       n_iter=3, cv=kfold, scoring='accuracy',
                                       error_score='raise', n_jobs=-1)
    random_search.fit(X_train_fold, y_train_fold)

    # Obtendo o melhor modelo após a busca aleatória
    best_model = random_search.best_estimator_

    # Realizando previsões no conjunto de validação
    y_val_pred = best_model.predict(X_val_fold)

  # Calcule métricas para esta dobra
    auc, acc, cf, precision, recall, f1 = calculate_metrics(y_val_fold, y_val_pred)

    # Armazene métricas
    auc_scores.append(auc)
    accuracy_scores.append(acc)
    confusion_matrices.append(cf)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

print("Terminou")

In [ ]:
# Imprimindo os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(random_search.best_params_)



In [ ]:
# Calculando média das métricas ao longo de todas as dobras
mean_auc = np.mean(auc_scores)
mean_accuracy = np.mean(accuracy_scores)
mean_confusion_matrix = np.mean(confusion_matrices, axis=0)
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)
mean_f1 = np.mean(f1_scores)

# Plotando as métricas
plt.figure(figsize=(12, 8))

# Plot da Curva ROC
plt.subplot(2, 2, 1)
mean_fpr, mean_tpr, _ = roc_curve(y_val_fold, y_val_pred)
plt.plot(mean_fpr, mean_tpr, color='b', lw=2, label=f'Mean ROC (AUC = {mean_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC Média')
plt.legend(loc='lower right')

# Plot da Acurácia
plt.subplot(2, 2, 2)
accuracy_values = np.array(accuracy_scores)
plt.bar(range(len(accuracy_values)), accuracy_values, color='g', alpha=0.7)
plt.xlabel('Dobra da Validação Cruzada')
plt.ylabel('Acurácia')
plt.title(f'Acurácia em cada Dobra (Média = {mean_accuracy:.2f})')

# Plot da Precisão (Precision)
plt.subplot(2, 2, 3)
precision_values = np.array(precision_scores)
plt.bar(range(len(precision_values)), precision_values, color='r', alpha=0.7)
plt.xlabel('Dobra da Validação Cruzada')
plt.ylabel('Precisão')
plt.title(f'Precisão em cada Dobra (Média = {mean_precision:.2f})')

# Plot do Recall
plt.subplot(2, 2, 4)
recall_values = np.array(recall_scores)
plt.bar(range(len(recall_values)), recall_values, color='m', alpha=0.7)
plt.xlabel('Dobra da Validação Cruzada')
plt.ylabel('Recall')
plt.title(f'Recall em cada Dobra (Média = {mean_recall:.2f})')

plt.tight_layout()
plt.show()

# Matriz de Confusão Média
print("Média da Matriz de Confusão:")
print(mean_confusion_matrix)

# Métrica F1-Score
print(f"Média do F1-Score: {mean_f1:.2f}")